In [1]:
import numpy as np
import time
from scipy import optimize
import matplotlib.pyplot as plt

In [28]:
starttime = time.time()

# Fixed parameters
filename = "n07_Un1Vn25.csv"
file = open(filename,"w+")
K = 0.0
U = -1.0
V = -2.5
Temp = 0.03
nval = 0.7
Lsmall = 12
Llarge = 40
runtime = 6*60*60
# Fixed functions
def eps(kx,ky):
    f = -2.0*(np.cos(kx) + np.cos(ky))
    return f

def sk(kx, ky):
    f = 1/2*(np.cos(kx) + np.cos(ky))
    return f

def dk(kx, ky):
    f = 1/2*(np.cos(kx) - np.cos(ky))
    return f

def deltaSing(kx,ky,d0,ds,dd):
    sing = (d0 + ds*sk(kx,ky) + dd*dk(kx,ky))
    return sing

def deltaTrip(kx,ky,dx,dy):
    trip = dx*np.sin(kx) + dy*np.sin(ky) #can't alter this phase for mixed singlet triplet
    return trip

def energy(mu,kx,ky,d0,ds,dd,dx,dy):
    f = np.sqrt((eps(kx, ky) - mu)**2 + np.abs(deltaSing(kx,ky,d0,ds,dd))**2+np.abs(deltaTrip(kx,ky,dx,dy))**2)
    return f

def fermi(T,mu,kx,ky,d0,ds,dd,dx,dy):
    f = 1.0/(np.exp(energy(mu,kx,ky,d0,ds,dd,dx,dy)/T) + 1.0)
    return f

def g(T,mu,kx,ky,d0,ds,dd,dx,dy):
    f = (1.0 - 2.0*fermi(T,mu,kx,ky,d0,ds,dd,dx,dy))/(2.0*energy(mu,kx,ky,d0,ds,dd,dx,dy)+0.0000001)
    return f

In [8]:
#    nval = 0.6
    
#    L = 32
#    NSites = L**2
#    delK = 2.0*np.pi/np.sqrt(NSites)
#    kvals = [-np.pi + delK + i*delK for i in range(L)]
    
#    def n(mu,T,d0,ds,dd,dx,dy):
#        sump = 2.0/(NSites)*sum((eps(kvals[i],kvals[j])-mu)*g(T,mu,kvals[i],kvals[j],d0,ds,dd,dx,dy) for i in range(len(kvals)) for j in range(len(kvals)))
#        f = 1.0-sump
#        return f

#    sktab = np.zeros((len(kvals),len(kvals)))
#    dktab = np.zeros((len(kvals),len(kvals)))
#    xtab = np.zeros((len(kvals),len(kvals)))
#    ytab = np.zeros((len(kvals),len(kvals)))
#    for i in range(len(kvals)):
#        for j in range(int(len(kvals))):
#            sktab[i,j] = sk(kvals[i], kvals[j])
#            dktab[i,j] = dk(kvals[i], kvals[j])
#            xtab[i,j] = np.sin(kvals[i])
#            ytab[i,j] = np.sin(kvals[j])
            
#    def numeqn(mu,d0,ds,dd,dx,dy):
#        return nval - n(mu,Temp,d0,ds,dd,dx,dy)
        
#    def FreeE(p):
#        Del0, Dels, Deld, Delx, Dely = p 
#    
#        mu = optimize.root(numeqn, -1.5, args=(Del0,Dels,Deld,Delx,Dely), method='broyden1', options={'ftol':0.01}).x
    
#        gp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
#        lnp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
#        epsEp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
#        delSingp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
#        delTripp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
#        del2p = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
#        i00 = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
#        iss = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
#        idd = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
#        ixx = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
#        iyy = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
#        for i in range(len(kvals)):
#            for j in range(len(kvals)):
#                gp[i,j] = g(Temp, mu, kvals[i], kvals[j], Del0, Dels, Deld, Delx, Dely)
#                lnp[i,j] = np.log(1.0-fermi(Temp, mu, kvals[i], kvals[j], Del0, Dels, Deld, Delx, Dely))
#                epsEp[i,j] = eps(kvals[i],kvals[j])-mu-energy(mu,kvals[i],kvals[j],Del0,Dels,Deld,Delx,Dely)
#                delSingp[i,j] = deltaSing(kvals[i],kvals[j],Del0,Dels,Deld)
#                delTripp[i,j] = deltaTrip(kvals[i],kvals[j],Delx,Dely)
#                del2p[i,j] = (np.abs(delSingp[i,j]+delTripp[i,j])**2)*gp[i,j]
#                i00[i,j] = delSingp[i,j]*gp[i,j]
#                iss[i,j] = sktab[i,j]*i00[i,j]
#                idd[i,j] = dktab[i,j]*i00[i,j]
#                ixx[i,j] = xtab[i,j]*(delTripp[i,j]*gp[i,j])
#                iyy[i,j] = ytab[i,j]*(delTripp[i,j]*gp[i,j])
            
#        I00 = np.abs(1.0/(NSites)*np.sum(i00))**2
#        Iss = np.abs(1.0/(NSites)*np.sum(iss))**2
#        Idd = np.abs(1.0/(NSites)*np.sum(idd))**2
#        Ixx = np.abs(1.0/(NSites)*np.sum(ixx))**2
#        Iyy = np.abs(1.0/(NSites)*np.sum(iyy))**2

#        logterm = 2.0*Temp/NSites*(np.sum(lnp))
#        enerterm = 1.0/(NSites)*(np.sum(epsEp))+2.0/(NSites)*(np.sum(del2p))
#        Vterm = U*I00 + 4.0*V*Iss +4.0*V*Idd + 2.0*V*Ixx + 2.0*V*Iyy
    
#        return np.real(enerterm + logterm + mu*nval + Vterm)
    
#print(FreeE((0.0,0.0,0.1*16,0.15*8,0.0)))
#print(FreeE((0.02*16,0.0,0.1*16,0.15*8,0.0)))
#print(FreeE((0.0,0.02*16,0.1*16,0.15*8,0.0)))
#print(FreeE((0.02*16,0.02*16,0.1*16,0.15*8,0.0)))
#print(FreeE((0.04*16,0.0,0.1*16,0.15*8,0.0)))
#print(FreeE((0.0,0.04*16,0.1*16,0.15*8,0.0)))
#print(FreeE((0.04*16,0.04*16,0.1*16,0.15*8,0.0)))
#print(FreeE((0.04*16,0.02*16,0.1*16,0.15*8,0.0)))
#print(FreeE((0.02*16,0.04*16,0.1*16,0.15*8,0.0)))
#print(FreeE((0.01963999342215773, 0.27494382454315874, 1.6275162251029844, 1.2189350085231294, 2.923524236852042e-07)))

In [32]:
Temp = 0.24
U = -1.0
V = -2.5
Tempend = 0.6
while V<-2.48:
    U = -1.0
    while Temp<Tempend:
        print('nval =', nval,' Temp =', Temp, ' U = ',U, 'V = ',V)
        L = Lsmall
        NSites = L**2
        delK = 2.0*np.pi/np.sqrt(NSites)
        kvals = [-np.pi + delK + i*delK for i in range(L)]

        def n(mu,T,d0,ds,dd,dx,dy):
            sump = 2.0/(NSites)*sum((eps(kvals[i],kvals[j])-mu)*g(T,mu,kvals[i],kvals[j],d0,ds,dd,dx,dy) for i in range(len(kvals)) for j in range(len(kvals)))
            f = 1.0-sump
            return f

        sktab = np.zeros((len(kvals),len(kvals)))
        dktab = np.zeros((len(kvals),len(kvals)))
        xtab = np.zeros((len(kvals),len(kvals)))
        ytab = np.zeros((len(kvals),len(kvals)))
        for i in range(len(kvals)):
            for j in range(int(len(kvals))):
                sktab[i,j] = sk(kvals[i], kvals[j])
                dktab[i,j] = dk(kvals[i], kvals[j])
                xtab[i,j] = np.sin(kvals[i])
                ytab[i,j] = np.sin(kvals[j])

        def numeqn(mu,d0,ds,dd,dx,dy):
            return nval - n(mu,Temp,d0,ds,dd,dx,dy)

        def FreeE(p):
            Del0, Dels, Deld, Delx, Dely = p 

            mu = optimize.root(numeqn, -1.5, args=(Del0,Dels,Deld,Delx,Dely), method='broyden1', options={'ftol':0.02}).x

            gp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            lnp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            epsEp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            delSingp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            delTripp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            del2p = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            i00 = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            iss = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            idd = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            ixx = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            iyy = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            for i in range(len(kvals)):
                for j in range(len(kvals)):
                    gp[i,j] = g(Temp, mu, kvals[i], kvals[j], Del0, Dels, Deld, Delx, Dely)
                    lnp[i,j] = np.log(1.0-fermi(Temp, mu, kvals[i], kvals[j], Del0, Dels, Deld, Delx, Dely))
                    epsEp[i,j] = eps(kvals[i],kvals[j])-mu-energy(mu,kvals[i],kvals[j],Del0,Dels,Deld,Delx,Dely)
                    delSingp[i,j] = deltaSing(kvals[i],kvals[j],Del0,Dels,Deld)
                    delTripp[i,j] = deltaTrip(kvals[i],kvals[j],Delx,Dely)
                    del2p[i,j] = (np.abs(delSingp[i,j]+delTripp[i,j])**2)*gp[i,j]
                    i00[i,j] = delSingp[i,j]*gp[i,j]
                    iss[i,j] = sktab[i,j]*i00[i,j]
                    idd[i,j] = dktab[i,j]*i00[i,j]
                    ixx[i,j] = xtab[i,j]*(delTripp[i,j]*gp[i,j])
                    iyy[i,j] = ytab[i,j]*(delTripp[i,j]*gp[i,j])

            I00 = np.abs(1.0/(NSites)*np.sum(i00))**2
            Iss = np.abs(1.0/(NSites)*np.sum(iss))**2
            Idd = np.abs(1.0/(NSites)*np.sum(idd))**2
            Ixx = np.abs(1.0/(NSites)*np.sum(ixx))**2
            Iyy = np.abs(1.0/(NSites)*np.sum(iyy))**2

            logterm = 2.0*Temp/NSites*(np.sum(lnp))
            enerterm = 1.0/(NSites)*(np.sum(epsEp))+2.0/(NSites)*(np.sum(del2p))
            Vterm = U*I00 + 4.0*V*Iss +4.0*V*Idd + 2.0*V*Ixx + 2.0*V*Iyy

            return np.real(enerterm + logterm + mu*nval + Vterm)

        def FreeEMF(p):
            Del0, Dels, Deld, Delx, Dely = p 

            mu = optimize.root(numeqn, -1.5, args=(Del0,Dels,Deld,Delx,Dely), method='broyden1', options={'ftol':0.01}).x

            gp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            lnp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            epsEp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            delSingp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            delTripp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            del2p = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            i00 = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            iss = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            idd = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            ixx = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            iyy = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            for i in range(len(kvals)):
                for j in range(len(kvals)):
                    gp[i,j] = g(Temp, mu, kvals[i], kvals[j], Del0, Dels, Deld, Delx, Dely)
                    lnp[i,j] = np.log(1.0-fermi(Temp, mu, kvals[i], kvals[j], Del0, Dels, Deld, Delx, Dely))
                    epsEp[i,j] = eps(kvals[i],kvals[j])-mu-energy(mu,kvals[i],kvals[j],Del0,Dels,Deld,Delx,Dely)
                    delSingp[i,j] = deltaSing(kvals[i],kvals[j],Del0,Dels,Deld)
                    delTripp[i,j] = deltaTrip(kvals[i],kvals[j],Delx,Dely)
                    del2p[i,j] = (np.abs(delSingp[i,j]+delTripp[i,j])**2)*gp[i,j]
                    i00[i,j] = delSingp[i,j]*gp[i,j]
                    iss[i,j] = sktab[i,j]*i00[i,j]
                    idd[i,j] = dktab[i,j]*i00[i,j]
                    ixx[i,j] = xtab[i,j]*(delTripp[i,j]*gp[i,j])
                    iyy[i,j] = ytab[i,j]*(delTripp[i,j]*gp[i,j])

            I00 = np.abs(1.0/(NSites)*np.sum(i00))**2
            Iss = np.abs(1.0/(NSites)*np.sum(iss))**2
            Idd = np.abs(1.0/(NSites)*np.sum(idd))**2
            Ixx = np.abs(1.0/(NSites)*np.sum(ixx))**2
            Iyy = np.abs(1.0/(NSites)*np.sum(iyy))**2

            logterm = 2.0*Temp/NSites*(np.sum(lnp))
            enerterm = 1.0/(NSites)*(np.sum(epsEp))+2.0/(NSites)*(np.sum(del2p))
            #Vterm = U*I00 + 4.0*V*Iss +4.0*V*Idd + 2.0*V*Ixx + 2.0*V*Iyy

            return np.real(enerterm + logterm + mu*nval)

        def FreeEs(p):
            Del0, Dels = p 
            Deld = 0.0
            Delx = 0.0
            Dely = 0.0
            return FreeE((Del0,Dels,Deld,Delx,Dely))

        def FreeEd(p):
            Deld = p 
            Del0 = 0.0
            Dels = 0.0
            Delx = 0.0
            Dely = 0.0
            return FreeE((Del0,Dels,Deld,Delx,Dely))

        def FreeEp(p):
            Delx = p 
            Del0 = 0.0
            Dels = 0.0
            Deld = 0.0
            Dely = 0.0
            return FreeE((Del0,Dels,Deld,Delx,Dely))

        def FreeEpy(p):
            Dely = p 
            Del0 = 0.0
            Dels = 0.0
            Deld = 0.0
            Delx = 0.0
            return FreeE((Del0,Dels,Deld,Delx,Dely))

        freeEval = 10.0;
        for i in range(15):
            r1 = 0.0
            r2 = 0.0
            r3 = np.random.uniform(V/100.0,V/3.0)
            r4 = 0.0
            r5 = 0.0
            f = FreeE((r1,r2,r3,r4,r5))
            if f<freeEval:
                freeEval = f
                globmin = np.array([r3])
                print(globmin,f, FreeEMF((r1,r2,r3,r4,r5)))

       # globmind=optimize.minimize(FreeEd, globmin, callback=None, options={'gtol': 2e-3, 'disp': True}).x 
        globmind1=np.array([0.0,0.0,globmin[0],0.015,0.00])
        globmind1=optimize.minimize(FreeE, globmind1, callback=None, options={'gtol': 2e-3, 'disp': True}).x
        freeEdval1=FreeE(globmind1)

        globmind2=np.array([0.015,0.015,globmin[0],0.0,0.0])
        globmind2=optimize.minimize(FreeE, globmind2, callback=None, options={'gtol': 2e-3, 'disp': True}).x
        freeEdval2=FreeE(globmind2)

        globmind3=np.array([0.015,0.015,globmin[0],0.015,0.0])
        globmind3=optimize.minimize(FreeE, globmind3, callback=None, options={'gtol': 2e-3, 'disp': True}).x
        freeEdval3=FreeE(globmind3)

        freeEval = 10.0;
        for i in range(78):
            r1 = np.random.uniform(U/100.0,U/3.0)
            r2 = np.random.uniform(-V/3.0,V/3.0)
            r3 = 0.0
            r4 = 0.0
            r5 = 0.0
            f = FreeE((r1,r2,r3,r4,r5))
            if f<freeEval:
                freeEval = f
                globmin = np.array([r1,r2])
                print(globmin,f,FreeEMF((r1,r2,r3,r4,r5)))

        #globmins=optimize.minimize(FreeEs, globmin, callback=None, options={'gtol': 1e-3, 'disp': True}).x
        globmins1=np.array([globmin[0],globmin[1],0.04,0.00,0.00])
        globmins1=optimize.minimize(FreeE, globmins1, callback=None, options={'gtol': 1e-3, 'disp': True}).x
        freeEsval1=FreeE(globmins1)

        globmins2=np.array([globmin[0],globmin[1],0.00,0.02,0.00])
        globmins2=optimize.minimize(FreeE, globmins2, callback=None, options={'gtol': 1e-3, 'disp': True}).x
        freeEsval2=FreeE(globmins2)

        globmins3=np.array([globmin[0],globmin[1],0.02,0.02,0.00])
        globmins3=optimize.minimize(FreeE, globmins3, callback=None, options={'gtol': 1e-3, 'disp': True}).x
        freeEsval3=FreeE(globmins3)

        freeEval = 10.0;
        for i in range(78):
            r1 = 0.0
            r2 = 0.0
            r3 = 0.0
            r4 = np.random.uniform(V/100.0,V/3.0)
            r5 = 0.0
            f = FreeE((r1,r2,r3,r4,r5))
            if f<freeEval:
                freeEval = f
                globmin = np.array([r4])
                print(globmin,f,FreeEMF((r1,r2,r3,r4,r5)))

        #globminp=optimize.minimize(FreeEp, globmin, callback=None, options={'gtol': 4e-3, 'disp': True}).x
        globminp1=np.array([0.02,0.02,0.0,globmin[0],0.0])
        globminp1=optimize.minimize(FreeE, globminp1, callback=None, options={'gtol': 4e-3, 'disp': True}).x
        freeEpval1=FreeE(globminp1)

        globminp2=np.array([0.0,0.0,0.02,globmin[0],0.0])
        globminp2=optimize.minimize(FreeE, globminp2, callback=None, options={'gtol': 4e-3, 'disp': True}).x
        freeEpval2=FreeE(globminp2)

        globminp3=np.array([0.02,0.02,0.02,globmin[0],0.0])
        globminp3=optimize.minimize(FreeE, globminp3, callback=None, options={'gtol': 4e-3, 'disp': True}).x
        freeEpval3=FreeE(globminp3)


        freelist=np.array([freeEdval1,freeEdval2,freeEdval3,freeEsval1,freeEsval2,freeEsval3,freeEpval1,freeEpval2,freeEpval3])
        globminlist=np.array(np.array([globmind1,globmind2,globmind3,globmins1,globmins2,globmins3,globminp1,globminp2,globminp3]))
        result = np.where(freelist == np.amin(freelist))
        globmin = globminlist[result[0]][0]
        print(globmin, " ", np.amin(freelist))

        # Fixed parameters for larger lattice
        L = Llarge
        NSites = L**2
        delK = 2.0*np.pi/np.sqrt(NSites)
        kvals = [-np.pi + delK + i*delK for i in range(L)]

        def n(mu,T,d0,ds,dd,dx,dy):
            sump = 2.0/(NSites)*sum((eps(kvals[i],kvals[j])-mu)*g(T,mu,kvals[i],kvals[j],d0,ds,dd,dx,dy) for i in range(len(kvals)) for j in range(len(kvals)))
            f = 1.0-sump
            return f

        sktab = np.zeros((len(kvals),len(kvals)))
        for i in range(len(kvals)):
            for j in range(int(len(kvals))):
                sktab[i,j] = sk(kvals[i], kvals[j])

        dktab = np.zeros((len(kvals),len(kvals)))
        for i in range(len(kvals)):
            for j in range(int(len(kvals))):
                dktab[i,j] = dk(kvals[i], kvals[j])

        xtab = np.zeros((len(kvals),len(kvals)))
        for i in range(len(kvals)):
            for j in range(int(len(kvals))):
                xtab[i,j] = (np.sin(kvals[i]))

        ytab = np.zeros((len(kvals),len(kvals)))
        for i in range(len(kvals)):
            for j in range(int(len(kvals))):
                ytab[i,j] = (np.sin(kvals[j]))

        def FreeE(p):
            Del0, Dels, Deld, Delx, Dely = p 

            mu = optimize.root(numeqn, -1.5, args=(Del0,Dels,Deld,Delx,Dely), method='broyden1', options={'ftol':0.004}).x

            gp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            lnp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            epsEp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            delSingp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            delTripp = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            del2p = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            i00 = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            iss = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            idd = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            ixx = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            iyy = np.zeros((len(kvals),len(kvals)),dtype=np.complex_)
            for i in range(len(kvals)):
                for j in range(len(kvals)):
                    gp[i,j] = g(Temp, mu, kvals[i], kvals[j], Del0, Dels, Deld, Delx, Dely)
                    lnp[i,j] = np.log(1.0-fermi(Temp, mu, kvals[i], kvals[j], Del0, Dels, Deld, Delx, Dely))
                    epsEp[i,j] = eps(kvals[i],kvals[j])-mu-energy(mu,kvals[i],kvals[j],Del0,Dels,Deld,Delx,Dely)
                    delSingp[i,j] = deltaSing(kvals[i],kvals[j],Del0,Dels,Deld)
                    delTripp[i,j] = deltaTrip(kvals[i],kvals[j],Delx,Dely)
                    del2p[i,j] = (np.abs(delSingp[i,j]+delTripp[i,j])**2)*gp[i,j]
                    i00[i,j] = delSingp[i,j]*gp[i,j]
                    iss[i,j] = sktab[i,j]*i00[i,j]
                    idd[i,j] = dktab[i,j]*i00[i,j]
                    ixx[i,j] = xtab[i,j]*(delTripp[i,j]*gp[i,j])
                    iyy[i,j] = ytab[i,j]*(delTripp[i,j]*gp[i,j])

            I00 = np.abs(1.0/(NSites)*np.sum(i00))**2
            Iss = np.abs(1.0/(NSites)*np.sum(iss))**2
            Idd = np.abs(1.0/(NSites)*np.sum(idd))**2
            Ixx = np.abs(1.0/(NSites)*np.sum(ixx))**2
            Iyy = np.abs(1.0/(NSites)*np.sum(iyy))**2

            logterm = 2.0*Temp/NSites*(np.sum(lnp))
            enerterm = 1.0/(NSites)*(np.sum(epsEp))+2.0/(NSites)*(np.sum(del2p))
            Vterm = U*I00 + 4.0*V*Iss +4.0*V*Idd + 2.0*V*Ixx + 2.0*V*Iyy

            return np.real(enerterm + logterm + mu*nval + Vterm)

        globmin=optimize.minimize(FreeE, globmin, callback=None, options={'gtol': 3e-6, 'disp': True}).x
        F=FreeE(globmin)

        mu = optimize.root(numeqn, -1.5, args=(globmin[0],globmin[1],globmin[2],globmin[3],globmin[4]), method='broyden1', options={'ftol':0.008}).x
        data = (Temp, mu, globmin[0],globmin[1],globmin[2],globmin[3],globmin[4], F)
        print(data)
        file.write("%s,%s,%s,%s,%s,%s,%s,%s\n" % data)
        Temp = Temp + 10.0
        #print("elapsed time: ", (time.time()-starttime))
        #if (time.time()-starttime)/runtime > 0.9:
        #    Temp = Tempend
    V = V + 0.1
    
#file.close()

nval = 0.7  Temp = 0.24  U =  -1.0 V =  -2.5
[-0.06770646] -1.5569620004555775 -1.5560222910798007
[-0.29149586] -1.5612805826048393 -1.5452783743982683
[-0.47233421] -1.5664754265869942 -1.5295136581863813
[-0.73248147] -1.5724077790806685 -1.5019884300749804
Optimization terminated successfully.
         Current function value: -1.574631
         Iterations: 2
         Function evaluations: 28
         Gradient evaluations: 4
Optimization terminated successfully.
         Current function value: -1.574635
         Iterations: 7
         Function evaluations: 56
         Gradient evaluations: 8
Optimization terminated successfully.
         Current function value: -1.574635
         Iterations: 7
         Function evaluations: 56
         Gradient evaluations: 8
[-0.31149646  0.66680332] -1.527470190937995 -1.513419233240813
[-0.19928208  0.15254935] -1.5475638583521387 -1.541730817240028
[-0.10372931 -0.46317315] -1.5495056745905422 -1.5370097227112187
[-0.10156861  0.02126556] -1.55

In [33]:
file.close()